In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/u/shuhan/projects/vla')

In [56]:
import accelerate

accelerator = accelerate.Accelerator()

/u/shuhan/anaconda3/envs/vla_hw/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [57]:
accelerator.local_process_index

0

In [2]:
import pickle
import gymnasium
import highway_env
import random

goal_spec_dataset_path = '/u/shuhan/projects/vla/data/highway_env/lane_change_goal_spec_data.pkl'
with open(goal_spec_dataset_path, 'rb') as f:
  goal_spec_dataset = pickle.load(f)

def get_ego_lane_id(curr_obs):
  lane_cnt = 5
  lane_width = 1.0 / lane_cnt
  abs_y = curr_obs[..., 2].copy()
  abs_y[1:] += abs_y[:1]
  abs_y += lane_width / 2
  lane_ids = (abs_y / lane_width).astype(int)
  ego_lane_id = lane_ids[0]
  return ego_lane_id

In [3]:
import copy
from tqdm import tqdm

env_states = []
path_infos = []
sample_num = 1000

for i in tqdm(range(sample_num)):
  env = gymnasium.make("highway-fast-v0", render_mode='rgb_array', config={"lanes_count": 5})
  curr_obs, _ = env.reset()
  env_state_cache = copy.deepcopy(env.unwrapped.__dict__)
  ego_lane_id = get_ego_lane_id(curr_obs)

  sampled_path_info = random.choice(goal_spec_dataset[ego_lane_id])

  env_states.append(env_state_cache)
  path_infos.append(sampled_path_info)

data = {
  'env_states': env_states,
  'path_infos': path_infos
}


100%|██████████| 1000/1000 [00:26<00:00, 37.90it/s]


In [5]:
save_path = '/storage/Datasets/highway_env/highway_fast_v0_dqn_meta_action_5_lanes/rollouts_env/rollouts_env_states_1k.pkl'
with open(save_path, 'wb') as f:
  pickle.dump(data, f)

In [55]:
subset_data = copy.deepcopy(data)
subset_data['env_states'] = subset_data['env_states'][:8]
subset_data['path_infos'] = subset_data['path_infos'][:8]

subset_save_path = '/storage/Datasets/highway_env/highway_fast_v0_dqn_meta_action_5_lanes/rollouts_env/rollouts_env_states_8_sample.pkl'
with open(subset_save_path, 'wb') as f:
  pickle.dump(subset_data, f)


In [6]:
with open(save_path, 'rb') as f:
  dataset = pickle.load(f)

env_states = dataset['env_states']
path_infos = dataset['path_infos']


In [41]:
from src.environments.highway_env.dataset import HighwayEnvInitStateDataset

dataset = HighwayEnvInitStateDataset(save_path)


In [42]:
sample_idx = 15

env_state_cache, path_info = dataset[sample_idx]

In [45]:
def collate_fn_env_init_state(batch):
    env_states, path_infos = zip(*batch)
    return env_states, path_infos

In [53]:
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn_env_init_state)

for env_state_cache, path_info in dataloader:
  print(env_state_cache[0])
  print(path_info[0])

{'config': {'observation': {'type': 'Kinematics'}, 'action': {'type': 'DiscreteMetaAction'}, 'simulation_frequency': 5, 'policy_frequency': 1, 'other_vehicles_type': 'highway_env.vehicle.behavior.IDMVehicle', 'screen_width': 600, 'screen_height': 150, 'centering_position': [0.3, 0.5], 'scaling': 5.5, 'show_trajectories': False, 'render_agent': True, 'offscreen_rendering': False, 'manual_control': False, 'real_time_rendering': False, 'lanes_count': 5, 'vehicles_count': 20, 'controlled_vehicles': 1, 'initial_lane_id': None, 'duration': 30, 'ego_spacing': 1.5, 'vehicles_density': 1, 'collision_reward': -1, 'right_lane_reward': 0.1, 'high_speed_reward': 0.4, 'lane_change_reward': 0, 'reward_speed_range': [20, 30], 'normalize_reward': True, 'offroad_terminal': False}, 'road': [MDPVehicle #496: [118.39500474   0.        ], IDMVehicle #880: [138.27514654   8.        ], IDMVehicle #120: [155.29460546   8.        ], IDMVehicle #360: [173.51670611   4.        ], IDMVehicle #600: [192.25151131   

In [43]:


new_env = gymnasium.make("highway-fast-v0", render_mode='rgb_array', config={"lanes_count": 5})
_, _ = new_env.reset()
new_env.unwrapped.__dict__.update(copy.deepcopy(env_state_cache))
curr_obs = new_env.unwrapped.observation_type.observe()
ego_lane_id = get_ego_lane_id(curr_obs)

print(ego_lane_id)
print(curr_obs)
print(path_info)

curr_obs, _, _, _, _ = new_env.step(2)

print(curr_obs)


4
[[ 1.          0.5927052   0.8         0.3125      0.        ]
 [ 1.          0.08999801  0.         -0.0181632   0.        ]
 [ 1.          0.1893897  -0.4        -0.02505371  0.        ]
 [ 1.          0.2925397  -0.6        -0.01487328  0.        ]
 [ 1.          0.37674394 -0.6        -0.03288624  0.        ]]
{'goal_spec': 'Goal is to reach Lane 4. Need to go through path Lane 4 -> Lane 3 -> Lane 2 -> Lane 3 -> Lane 4.', 'hop_lane_ids': [4, 3, 2, 3, 4]}
[[ 1.          0.7177052   0.8         0.3125      0.        ]
 [ 1.          0.08116179 -0.03671233 -0.03861943 -0.08283388]
 [ 1.          0.178953   -0.4        -0.02743036  0.        ]
 [ 1.          0.27408892 -0.63897127 -0.0909245   0.02160919]
 [ 1.          0.3501971  -0.73719645 -0.0987533  -0.04119483]]
